In [6]:
import os
import cv2
import math
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC

import warnings; warnings.simplefilter('ignore')
SEED = 0

In [7]:
train_img_list, train_label_list = [],  []
test_img_list, test_label_list = [],  []

folder_dir = './Lab2/101_ObjectCategories'
for j, (root, dirs, file) in enumerate(os.walk(folder_dir)):

    for k, name in enumerate(dirs): 
        if name == "BACKGROUND_Google": # irrelevant class
            continue    
        
        filepath = folder_dir + "/" + name
        for i, f in enumerate(os.listdir(filepath)):
            if i > 80:
                break

            img_path = os.path.join(folder_dir, name, f)
            label = name
            
            if i < 30: # following the paper, we take 30 for train set and 50 for test. If there are are less than 50 for test, it doesnt matter (according to the paper)
                train_img_list.append(cv2.imread(img_path))
                train_label_list.append(label)
            else:
                test_img_list.append(cv2.imread(img_path))  
                test_label_list.append(label)

### Bag of Features (BoF)

In [8]:
# computes sift descriptor for imgs
def compute_sift_descriptor(imgs):
    
    step_size = 15
    sift_list = []
    for img in imgs:
        # sift = cv2.SIFT_create()
        sift = cv2.xfeatures2d.SIFT_create()
        kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, img.shape[0], step_size) 
                                        for x in range(0, img.shape[1], step_size)]
        
        dense_feat = sift.compute(img, kp)
        sift_list.append(dense_feat[1])
    return sift_list

def get_clusters(num_clusters, X_train_desc):
    k_means = KMeans(n_clusters=num_clusters, random_state=SEED).fit(X_train_desc)
    
    return k_means

def get_histogram(X_train, k_means_classifier, num_clusters):
    hist_list = []
    for i, img in enumerate(X_train):
        features = k_means_classifier.predict(img)
        hist = np.bincount(features, minlength = num_clusters).reshape(1,-1).flatten()
        hist_list.append(hist)
        
    return hist_list

In [10]:
X_train = compute_sift_descriptor(train_img_list)
X_test = compute_sift_descriptor(test_img_list)

X_train_desc = []
for i in range(len(X_train)):
    for j in range(X_train[i].shape[0]):
        X_train_desc.append(X_train[i][j,:])

X_train_desc = np.array(X_train_desc)

num_clusters = 60
k_means_classifier = get_clusters(num_clusters, X_train_desc)

X_train_hist = get_histogram(X_train, k_means_classifier, num_clusters)
X_test_hist = get_histogram(X_test, k_means_classifier, num_clusters)

  File "c:\Users\micha\miniconda3\envs\michael\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [11]:
# normalize histograms
scaler = StandardScaler().fit(X_train_hist)
train_hist = scaler.transform(X_train_hist)
test_hist = scaler.transform(X_test_hist)

# SVM
clf = LinearSVC(random_state=SEED)
clf.fit(train_hist, train_label_list)
pred = clf.predict(test_hist)
accuracy = np.mean(pred == test_label_list)*100

print(f"Classification Accuracy for BoF: {accuracy:.3f}%")

Classification Accuracy for BoF: 33.771%


### Spatial Pyramid Mapping (SPM)

In [64]:
# computes sift descriptor for imgs
def compute_sift_descriptor_grid(img):
    
    step_size = 2
    # sift = cv2.SIFT_create()
    sift = cv2.xfeatures2d.SIFT_create()
    kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, img.shape[0], step_size) 
                                    for x in range(0, img.shape[1], step_size)]
    
    dense_feat = sift.compute(img, kp)[1]

    return dense_feat

def SPM_features(img, L, k_means_classifier, num_clusters):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = compute_sift_descriptor_grid(img[y:y+h_step, x:x+w_step])                

                predict = k_means_classifier.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist

def get_SPM_histogram(X_train, L, k_means_classifier, num_clusters):
    hist_list = []
    for img in X_train:
        hist = SPM_features(img, L, k_means_classifier, num_clusters)
        hist_list.append(hist)
    
    return np.array(hist_list)

In [ ]:
    # height, width = img.shape[0], img.shape[1]
    
    # hist_list = []
    # for layer in range(L+1):
    #     h = math.floor(height/(2**layer))
    #     w = math.floor(width/(2**layer))
        
    #     for i in range(pow(2, layer)):
    #         for j in range(pow(2, layer)):
    #             features = compute_sift_descriptor_grid(img[j*h:(j+1)*h , i*h:(i+1)*w])
    #             pred = k_means_classifier.predict(features)
                
    #             hist = np.bincount(pred, minlength = num_clusters).reshape(1,-1).flatten()
    #             weight = pow(2,(1-L))
    #             weighted_hist = weight*hist
    #             hist_list.append(weighted_hist)
                
    # hist_list = np.array(hist_list).flatten()
    
    # standard_deviation = np.std(hist_list)
    # hist_list -= np.mean(hist_list)
    # hist_list /= standard_deviation
    
    # return hist_list

In [65]:
X_train_hist = get_SPM_histogram(train_img_list, 2, k_means_classifier, num_clusters)
X_test_hist = get_SPM_histogram(test_img_list, 2, k_means_classifier, num_clusters)

# train SVM
clf = LinearSVC(random_state=SEED)
clf.fit(X_train_hist, train_label_list)
pred = clf.predict(X_test_hist)
accuracy = np.mean(pred == test_label_list)*100

print(f"Classification Accuracy for BoF: {accuracy:.3f}%")

Classification Accuracy for BoF: 47.845%
